# Imports and Configurations

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [ ]:
sc = SparkContext()

In [ ]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [ ]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5555)

In [ ]:
lines = socket_stream.window(20)

In [ ]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [ ]:
# Get to 10 hashtags associated with search key
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word, creates the mapper
  .reduceByKey( lambda a, b: a + b ) # Reduces to sum of observations of a hashtag
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).registerTempTable("tweets") ) ) # Registers to a table.

__________
# TweetRead.py is now running and serving tweets 
__________

In [ ]:
ssc.start()    

In [ ]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [ ]:
import pandas as pd

In [ ]:
df = None
count = 0

while count < 50:
    
    # Plot the Top 10 hashtags of tweets that mention running (or any other keyword)
    time.sleep( 3 )
    top_10_hashtags = sqlContext.sql( 'SELECT tag, count FROM tweets' )
    top_10_df_current = top_10_hashtags.toPandas()

    # Use Top 10 at first run, then top 10 of all the past previous
    if df is None:
        df = top_10_df_current
    else:
        df = pd.concat([df,top_10_df_current])

    # A dynamic barplot of the count of each popular tag    
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="tag", data=df)
    plt.show()
    count = count + 1

In [ ]:
ssc.stop()